# Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV

# Import Data

In [ ]:
wine_data = pd.read_csv("winequality-red.csv")

# Describe Data

The data consists of 11 features and one target (output) value. The input features based on physicochemical tests are as follows:

1 - **fixed acidity** - The term refers to the amount of acids which do not evaporate easily during fermentation and aging and end up in the final bottled wine. It is a positive real number measured in grams per liter (g/L). Usually, red wines have fixed acidity values between 4 g/L to 7 g/L, however this range is not strictly fixed.

2 - **volatile acidity** - This feature is the counterpart of fixed acidity, however in this case it measures the amount of volatile acids in wine which are easily evaporating during fermentation and aging. Measured in g/L, usual values are between 0.3 g/L to 0.7 g/L.

3 - **citric acid** - The amount of citric acid in wine, measured in g/L. Usual values range from 0 g/L to 1 g/L.

4 - **residual sugar** - The amount of sugar left in wine after the fermentation process. It is used to balance acidity and makes red wine categorized into dry, semi-sweet and sweet variants. Once again, it is measured in g/L.

5 - **chlorides** - The amount of chloride salts in red wine (typically NaCl). Measured in g/L, typical values range from 0.012 g/L to 0.1 g/L.

6 - **free sulfur dioxide** - Free sulfur dioxide (SO₂) in wine is the portion of sulfur dioxide that is not bound to other molecules and is active as an antimicrobial and antioxidant. It is measured in milligrams per liter (mg/L) and has typical values from 1 mg/L to 30 mg/L.

7 - **total sulfur dioxide** - Total sulfur dioxide in wine includes both free and bound forms of SO₂. Once again, it is measured in mg/L. Typical values range from 10 mg/L to 150 mg/L.

8 - **density** - The density feature shows how close the wine molecules are located to each other, it measures how heavier or lighter the wine is compared to water. Density for wines is measure in grams per cubic centimeter (g/cm³). Typical values range from 0.99 g/cm³ to 1.003 g/cm³.

9 - **pH** - This feature measures the concentration of positive hydrogen ions in wine, which in simple terms shows the acidity of wine. pH values range from 1 to 13, where a pH value of 7 means the substance in completely neutral, lower values indicate acidic environment, and higher values indicate a basic one. The feature has no measure of units. Typical values fall into the range from 3.2 to 3.6.

10 - **sulphates** - The amount of sulphates in wine measured in g/L. Typical value range is from 0.3 g/L to 1 g/L.

11 - **alcohol** - Finally, the last feature shows the percentage of alcohol (ethanol content) present in wine. It is measured in percents and has typical values between 12% to 14.5%.

The output value is the **wine quality score**, which is an **integer** from 0 to 10. Lower scores represent poor wine quality, while high scores represent excellent quality.

Find more information about the dataset and it's origins [here](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).

# Exploratory Data Analysis (EDA)

In [ ]:
# Satine Aghababyan
import matplotlib.pyplot as plt
import seaborn as sns

df = wine_data

print("\nFirst few lines for display")
print(df.head())

print("\nSummary Statistics")
print(df.describe())

print("\nMissing Values")
print(df.isnull().sum())

print("\nDistribution of Features")
numeric_features = df.columns[:-1]
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_features, 1):
    plt.subplot(3, 4, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

#Էս մի հատ նայեք՝ պետք ա թե չէ
print("\nBoxplot Distribution of Features (for detecting outliers)")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_features, 1):
    plt.subplot(3, 4, i)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

#Էս փակագծերում գրածներս կարանք քոմենթ սարքենք եսլի չտո, կամ հանենք present անելուց կասենք
print("\nCount of Quality (for understanding if our data is balanced or biased)")
plt.figure(figsize=(6, 4))
sns.countplot(x='quality', data=df)
plt.title('Wine Quality Distribution')
plt.show()

print("\nCorrelation")
plt.figure(figsize=(12, 10))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

print("\nFeature vs Quality")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_features, 1):
    plt.subplot(3, 4, i)
    sns.boxplot(x='quality', y=col, data=df)
    plt.title(f'{col} vs Quality')
plt.tight_layout()
plt.show()

print("\nMissing Vales Count")
total_rows = len(df)
missing_counts = df.isnull().sum()
present_counts = total_rows - missing_counts
columns = df.columns

fig, ax = plt.subplots(figsize=(12, 6))

bars_present = ax.bar(columns, present_counts, color='skyblue', edgecolor='black', label='Present')

bars_missing = ax.bar(columns, missing_counts, bottom=present_counts, color='salmon',
                      edgecolor='black', hatch='///', label='Missing')

ax.set_ylabel('Number of Values')
ax.set_title('Present vs Missing Values per Feature')
plt.xticks(rotation=45)
ax.legend()

plt.tight_layout()
plt.show()


print("\nMean Feature Values per Wine Quality")
grouped_means = df.groupby('quality').mean()

grouped_means_T = grouped_means.T

plt.figure(figsize=(14, 8))
grouped_means_T.plot(kind='bar', figsize=(14, 8), colormap='viridis')

plt.title('Mean Feature Values by Wine Quality')
plt.ylabel('Mean Value')
plt.xlabel('Features')
plt.xticks(rotation=45)
plt.legend(title='Quality', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Data preprocessing

In [15]:
# Tigran Fahradyan
data = wine_data.to_numpy()

CV = 5

def split_data(data, seed=42):
    np.random.seed(seed)
    indices = np.random.permutation(len(data))
    
    train_end = int(0.9 * len(data))

    train = data[indices[:train_end]]
    test = data[indices[train_end:]]

    return train, test

def replace_nans(train, test):
    train_mean = np.nanmean(train[:, :-1], axis=0)
    for dataset in [train, test]:
        for i in range(dataset.shape[1] - 1):
            nan_mask = np.isnan(dataset[:, i])
            dataset[nan_mask, i] = train_mean[i]
    return train, test

train, test = split_data(data, seed=42)
train, test = replace_nans(train, test)

X_train, y_train = train[:, :-1], train[:, -1].astype(int)
X_test, y_test = test[:, :-1], test[:, -1].astype(int)

# Modelling

In [ ]:
class NaiveBayes:
    def __init__(self, n_bins=10):
        self.n_bins = n_bins

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.classes = np.unique(y)
        self.K = len(self.classes)
        self.N = len(y)

        self.class_counts = {c: np.sum(y == c) for c in self.classes}
        self.priors = {
            c: (self.class_counts[c] + 1) / (self.N + self.K)
            for c in self.classes
        }

        self.bins = []
        self.X_binned = np.zeros_like(X, dtype=int)
        for i in range(self.n_features):
            col, edges = pd.cut(X[:, i], bins=self.n_bins, retbins=True, labels=False, duplicates='drop')
            self.X_binned[:, i] = col
            self.bins.append(edges)

        self.feature_counts = {
            c: np.zeros((self.n_features, self.n_bins), dtype=int)
            for c in self.classes
        }

        for x, label in zip(self.X_binned, y):
            for i, bin_idx in enumerate(x):
                self.feature_counts[label][i][bin_idx] += 1

    def _predict_sample(self, x):
        x_binned = []
        for i, val in enumerate(x):
            bin_idx = np.digitize(val, self.bins[i]) - 1
            bin_idx = min(max(bin_idx, 0), self.n_bins - 1)
            x_binned.append(bin_idx)

        best_class = None
        max_log_prob = -np.inf

        for c in self.classes:
            log_prob = np.log(self.priors[c])

            for i, bin_idx in enumerate(x_binned):
                count = self.feature_counts[c][i][bin_idx]
                total = self.class_counts[c]
                prob = (count + 1) / (total + self.K)
                log_prob += np.log(prob)

            if log_prob > max_log_prob:
                max_log_prob = log_prob
                best_class = c

        return best_class

    def predict(self, X):
        return np.array([self._predict_sample(x) for x in X])

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

nb = NaiveBayes(n_bins=10)
nb.fit(X_train, y_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB(var_smoothing=1e-9)
gnb.fit(X_train, y_train)

gnb_val_preds = gnb.predict(X_val)
gnb_test_preds = gnb.predict(X_test)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

hparams = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"],
}

decision_tree = DecisionTreeClassifier(random_state=42)
grid_search_decision_tree = GridSearchCV(
    decision_tree,
    param_grid=hparams,
    scoring="accuracy",
    cv=CV
)
grid_search_decision_tree.fit(X_train, y_train);

dt = grid_search_decision_tree.best_estimator_

print(f"The best hparam configuration for decision tree is {grid_search_decision_tree.best_params_} with score {grid_search_decision_tree.best_score_:.4f}")

decision_tree_test_preds = grid_search_decision_tree.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

rf_test_preds = rf.predict(X_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(solver='svd')
lda.fit(X_train, y_train)

lda_test_preds = lda.predict(X_test)

In [ ]:
# kNN - Suren Hakobyan

# Gaussian Naive Bayes - Satine Aghababyan
# Logistic Regression - Suren Hakobyan
# Decision Tree - Tigran Fahradyan
# Random Forest - Satine Aghababyan
# SVM - Suren Hakobyan
# Neural Networks - Tigran Fahradyan
# LDA - Satine Aghababyan
# QDA - Suren Hakobyan


# Model Evaluation

In [ ]:
# հետո կերևա

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(name, model, X_test, y_test, is_custom=False):
    print(f"\n{name}")
    if is_custom:
        y_pred = model.predict(X_test)
    else:
        y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))


    all_classes = list(range(0, 11))
    cm = confusion_matrix(y_test, y_pred, labels=all_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()


evaluate_model("Naive Bayes", nb, X_test, y_test, is_custom=True)

evaluate_model("Gaussian Naive Bayes", gnb, X_test, y_test)

evaluate_model("Random Forest", rf, X_test, y_test)

evaluate_model("Linear Discriminant Analysis", lda, X_test, y_test)
